In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.optimizers import SGD

In [ ]:
import freezing
import resnet_sd

In [2]:
# Loading Data - SVHN - followed paper (https://arxiv.org/pdf/1603.09382.pdf)

# Imports
import numpy as np
import scipy.io


# Split train data into train and validation gettting certain number of labels from each class
def train_val_split_count(x_train, y_train, size, seed):
    
    if seed != None:
        np.random.seed(seed)  # Set seed if it is stated.

    labels = set(y_train.flatten())  # Get label names
    n_labels = len(labels)  # Get number of labels

    x_val = []
    y_val = []
    split = []

    
    for i in labels:
        labels_i = np.where(y_train == i)[0]  # Take set of only one label
        samples = np.random.choice(labels_i, size)  # TODO: Check if enough labels in the class
        split.append(samples)

    split = np.array(split).flatten()
    #print(split[:10])

    x_val = np.array(x_train[split])
    y_val = np.array(y_train[split])
    
    x_train = np.delete(x_train, split, axis=0)
    y_train = np.delete(y_train, split, axis=0)
    
    return (x_train, x_val, y_train, y_val)
def load_data_svhn(seed = None):
    

    # Load in MatLab matrices
    test_mat = scipy.io.loadmat('test_32x32.mat')
    train_mat = scipy.io.loadmat('train_32x32.mat')
    extra_mat = scipy.io.loadmat('extra_32x32.mat')


    # Get data from matrices
    x_test = test_mat.get('X')  #numpy arrays
    y_test = test_mat.get('y')

    x_train = train_mat.get('X')
    y_train = train_mat.get('y')

    x_extra = extra_mat.get('X')
    y_extra = extra_mat.get('y')
    

    # Reshape the matrices

    # [h,w,channels,samples] -> [samples,h,w,channels]
    
    x_test = np.transpose(x_test, axes=(3,0,1,2))
    x_train = np.transpose(x_train, axes=(3,0,1,2))
    x_extra = np.transpose(x_extra, axes=(3,0,1,2))


    # Split DATA
    x_train1, x_val1, y_train1, y_val1 = train_val_split_count(x_train, y_train, size = 400, seed = seed)
    x_extra2, x_val2, y_extra2, y_val2 = train_val_split_count(x_extra, y_extra, size = 200, seed = seed)


    # Add together train and extra data

    x_train_all = np.concatenate([x_train1, x_extra2])
    y_train_all = np.concatenate([y_train1, y_extra2])
    
    y_train_all -= 1  # So 0 would be smallest label and 9 biggest.
    #NB! Note that this way the labels are not actually correct, because 10 indicates the 0, FIX this.

    x_val_all = np.concatenate([x_val1, x_val2])
    y_val_all = np.concatenate([y_val1, y_val2])
    
    y_val_all -= 1  # So 0 would be smallest label and 9 biggest
    y_test -= 1  # So 0 would be smallest label and 9 biggest


    return ((x_train_all, y_train_all), (x_val_all, y_val_all), (x_test, y_test))

In [4]:
learning_rate = 0.1
nb_epochs = 50
batch_size = 128
nb_classes = 10
seed = 333
layers = 152  # n = 25 (152-2)/6

# data
print("Loading data, may take some time and memory!")
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_data_svhn(seed=seed)
print(x_train.shape)
print("Data loaded")


Loading data, may take some time and memory!
(598526, 32, 32, 3)
Data loaded


In [ ]:
def color_preprocessing(x_train, x_val, x_test):
    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')
    x_test = x_test.astype('float32')

    mean = np.mean(x_train, axis=(0, 1, 2))  # Per channel mean
    std = np.std(x_train, axis=(0, 1, 2))
    x_train = (x_train - mean) / std
    x_val = (x_val - mean) / std
    x_test = (x_test - mean) / std

    return x_train, x_val, x_test
x_train, x_val, x_test = color_preprocessing(x_train, x_val, x_test)  # Per channel mean

# Try with ImageDataGenerator, otherwise it takes massive amount of memory
img_gen = ImageDataGenerator(
    data_format="channels_last"
)

img_gen.fit(x_train)

y_train = np_utils.to_categorical(y_train, nb_classes)  # 1-hot vector
y_val = np_utils.to_categorical(y_val, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
model = resnet_sd.resnet_sd_model(img_shape=(32, 32), img_channels=3,
                        layers=layers, nb_classes=nb_classes, verbose=True)

In [6]:
freezing_list = []
for i in range(len(model.layers)):
  if i < len(model.layers) * 0.9:
    freezing_list.append(int(nb_epochs*0.6))
  elif i < len(model.layers) * 0.98:
    freezing_list.append(int(nb_epochs*0.96))
freezing_list.append(nb_epochs)
checkpointer = ModelCheckpoint('model_svhn_best_2_cont.hdf5', verbose=1, save_best_only=True)
sgd = SGD(learning_rate=0.1, momentum=0.9, nesterov=True) 

In [7]:
freezing.continue_training(model, img_gen, sgd, x_train, y_train, x_val, y_val, x_test, y_test,freezing_list,batch_size=batch_size,lr_schedule = [[0, 0.04],[2,0.1],[30,0.01],[35,0.001]],all_epochs=25,weights='resnet_sd_svhn_2_25.h5',cbks=[checkpointer], name='resnet_sd_svhn_2')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activation_3
.........vars
......activation_30
.........vars
......activation_31
.........vars
......activation_32
.........vars
......activa

2023-04-24 15:04:00.783989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200


4675/4675 [==============================] - ETA: 0s - loss: 0.1748 - accuracy: 0.9768
Epoch 1: val_loss improved from inf to 0.28792, saving model to model_svhn_best_2_cont.hdf5
4675/4675 [==============================] - 488s 98ms/step - loss: 0.1748 - accuracy: 0.9768 - val_loss: 0.2879 - val_accuracy: 0.9388 - lr: 0.1000
Epoch 2/5
4675/4675 [==============================] - ETA: 0s - loss: 0.1736 - accuracy: 0.9766
Epoch 2: val_loss improved from 0.28792 to 0.24030, saving model to model_svhn_best_2_cont.hdf5
4675/4675 [==============================] - 451s 96ms/step - loss: 0.1736 - accuracy: 0.9766 - val_loss: 0.2403 - val_accuracy: 0.9582 - lr: 0.1000
Epoch 3/5
4675/4675 [==============================] - ETA: 0s - loss: 0.1738 - accuracy: 0.9767
Epoch 3: val_loss did not improve from 0.24030
4675/4675 [==============================] - 448s 96ms/step - loss: 0.1738 - accuracy: 0.9767 - val_loss: 0.2565 - val_accuracy: 0.9502 - lr: 0.1000
Epoch 4/5
4675/4675 [================

[2.3778426551936036,
 0.005242779424509062,
 0.17868421971797943,
 0.0996491143236001,
 0.02129944383636701]